In [ ]:
import lightgbm as lgb
import numpy as np
import optuna
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import StratifiedKFold

# Define the objective function for Optuna optimization
def objective(trial):
    # Load dataset (replace this with your actual dataset loading)
    X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)

    # Split dataset into training and validation sets
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Define hyperparameters to tune
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    # Define empty list to store cross-validation scores
    scores = []

    # Perform cross-validation
    for train_index, valid_index in skf.split(X, y):
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]

        dtrain = lgb.Dataset(X_train, label=y_train)
        dvalid = lgb.Dataset(X_valid, label=y_valid)

        # Train LightGBM model
        model = lgb.train(params, dtrain, valid_sets=[dvalid], early_stopping_rounds=100, verbose_eval=False)

        # Predict validation set and calculate binary log loss
        y_pred = model.predict(X_valid)
        score = sklearn.metrics.log_loss(y_valid, y_pred)

        scores.append(score)

    # Return average binary log loss over all folds
    return np.mean(scores)

# Create study object and optimize hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Print best trial parameters and value
print('Best trial:')
trial = study.best_trial
print('  Value: {:.4f}'.format(trial.value))
print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))